<a href="https://colab.research.google.com/github/tbass134/GAN-pytorch/blob/main/GAN_Pytorch_Monet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import random


In [ ]:
!pip install wandb -q

import wandb
wandb.login()
# import wandb
# wandb.init(project='cifar10-gan', entity='tbass134')
# config = wandb.config

     |████████████████████████████████| 1.7 MB 7.3 MB/s 
     |████████████████████████████████| 142 kB 64.9 MB/s 
     |████████████████████████████████| 180 kB 71.2 MB/s 
     |████████████████████████████████| 97 kB 7.8 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#used when not using wandb
class Config():
  def __init__(self):
    pass
config = Config()

In [ ]:
class Critic(nn.Module): #aka discriminator
  def __init__(self, channels_img, features_d):
    super().__init__()
    self.critic = nn.Sequential(
      # input: N x channels_img x 64 x 64
      nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),
      nn.LeakyReLU(0.2),
      # _block(in_channels, out_channels, kernel_size, stride, padding)
      self._block(features_d, features_d * 2, 4, 2, 1),
      self._block(features_d * 2, features_d * 4, 4, 2, 1),
      self._block(features_d * 4, features_d * 8, 4, 2, 1),
      # After all _block img output is 4x4 (Conv2d below makes into 1x1)
      nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
    )
    
  def _block(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(
            in_channels, out_channels, kernel_size, stride, padding, bias=False,
        ),
        nn.InstanceNorm2d(out_channels, affine=True),
        nn.LeakyReLU(0.2),
    )

  def forward(self, x):
    return self.critic(x)

class Generator(nn.Module):
  def __init__(self, channels_noise, channels_img, features_g):
    super().__init__()
    self.gen = nn.Sequential(
    # Input: N x channels_noise x 1 x 1
      self._block(channels_noise, features_g * 16, 4, 1, 0),  # img: 4x4
      self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
      self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
      self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
      nn.ConvTranspose2d(
          features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
      ),
      # Output: N x channels_img x 64 x 64
      nn.Tanh(),
    )

  def _block(self, in_channels, out_channels, kernel_size, stride, padding):
      return nn.Sequential(
          nn.ConvTranspose2d(
              in_channels, out_channels, kernel_size, stride, padding, bias=False,
          ),
          nn.BatchNorm2d(out_channels),
          nn.ReLU(),
      )
       
  def forward(self, x):
    return self.gen(x)

In [ ]:
def initialize_weights(model):
    # Initializes weights according to the DCGAN paper
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d,  nn.InstanceNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)

In [ ]:
import glob
assert len(glob.glob("/content/drive/MyDrive/Kaggle Datasets/gan-getting-started/monet_jpg/*")) > 0

In [ ]:
class ImageDataset(Dataset):
    """ Custom Dataset class """
    def __init__(self, root: str, transform: transforms.Compose):
        self.root = root
        self.transform = transform
        self.all_imgs = tuple(os.path.join(root, p) for p in os.listdir(root) if p.endswith('.jpg'))

    def __len__(self) -> int:
        return len(self.all_imgs)

    def __getitem__(self, idx: int) -> torch.Tensor:
        photo_img = Image.open(self.all_imgs[idx])
        photo_img = self.transform(photo_img)
        return photo_img
            
    def plot_random_images(self, n_img: int) -> None:
        
        # Sample random number of images
        sampled_images = random.sample(self.all_imgs, n_img)
        
        images_list = list(np.transpose(torch.from_numpy(self.load_image(p)), (2, 0, 1)) for p in sampled_images)
        image_grid = torchvision.utils.make_grid(images_list)
        plt.figure(figsize=(20, 20))
        plt.imshow(np.transpose(image_grid, (1, 2, 0)))
        plt.axis('off')
        plt.show()
        
    
config.img_size = 64
config.img_channels = 3


tfs = transforms.Compose([
                transforms.Resize(config.img_size),
                transforms.ToTensor(),
                transforms.Normalize(
                [0.5 for _ in range(config.img_channels)], [0.5 for _ in range(config.img_channels)])

            ])

dataset = ImageDataset(root="/content/drive/MyDrive/Kaggle Datasets/gan-getting-started/monet_jpg/",
                           transform=tfs)


In [ ]:
# loader = DataLoader(dataset, batch_size=32, shuffle=True)
# next(enumerate(loader))

In [ ]:
def gradient_penalty(critic, real, fake, batch_size, device="cpu"):
    # print("real",real.shape)
    # print("fake",fake.shape)
    batch_size, C, H, W = real.shape
    alpha = torch.rand((batch_size, 1, 1, 1)).repeat(1, C, H, W).to(device)
    # print("alpha",alpha.shape)
    interpolated_images = real * alpha + fake * (1 - alpha)

    # Calculate critic scores
    mixed_scores = critic(interpolated_images)

    # Take the gradient of the scores with respect to the images
    gradient = torch.autograd.grad(
        inputs=interpolated_images,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
    return gradient_penalty

In [ ]:


def train(config=None):
  with wandb.init(config=config):
    # If called by wandb.agent, as below,
    # this config will be set by Sweep Controller
    config = wandb.config
    print(config)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    gen = Generator(config.noise_dim, config.img_channels, config.features_g).to(device)
    critic = Critic(config.img_channels, config.features_critic).to(device)
    initialize_weights(critic)
    initialize_weights(gen) 

    wandb.watch(gen, log="all")
    wandb.watch(critic, log="all")
    # print("Generator", gen)
    # print("Critic", critic)

    fixed_noise = torch.randn(32, config.noise_dim, 1, 1).to(device)

    loader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True)

    opt_critic = optim.Adam(critic.parameters(), lr=config.lr, betas=(0.5, 0.9))
    opt_gen = optim.Adam(gen.parameters(), lr=config.lr, betas=(0.5, 0.9))

    gen.train()
    critic.train()
    step = 0

    for epoch in range(config.num_epochs):

      print(f'epoch {epoch} of {config.num_epochs}')
      for batch_idx, real_images in enumerate(loader):
        print("batch_idx", batch_idx)
        real_images = real_images.to(device)
        cur_batch_size = real_images.shape[0]
        # train descriminator

        for _ in range(config.critic_iters):
          #generate noise
          noise = torch.randn(cur_batch_size, config.noise_dim, 1, 1).to(device)
          # print(noise.shape)
          fake_images = gen(noise)
          critic_real = critic(real_images).reshape(-1)
          critic_fake = critic(fake_images).reshape(-1)
          gp = gradient_penalty(critic, real_images, fake_images, config.batch_size, device)
          loss_critic = -(torch.mean(critic_real) - torch.mean(critic_fake)) + config.lambda_gp * gp
          critic.zero_grad()
          loss_critic.backward(retain_graph=True)
          opt_critic.step()

          # for p in critic.parameters():
          #   p.data.clamp_(-config.weight_clip, config.weight_clip)

        gen_fake = critic(fake_images).reshape(-1)
        loss_gen = -torch.mean(gen_fake)
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()


        if batch_idx % 5 == 0 and batch_idx > 0:
            gen.eval()
            critic.eval()
            
            losses = {"loss_critic": loss_critic, "loss_g":loss_gen}
            
            wandb.log({
                  "Epoch":f'{epoch}/{config.num_epochs}',
                  "Batch":f'{batch_idx}/{len(loader)}',
                  "Loss Critic":f'{loss_critic:.4f}',
                  "loss G":f'{loss_gen:.4f}'
              })

            with torch.no_grad():
                fake = gen(noise)

                
                img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)
                # plt.imshow(np.transpose(img_grid_fake.cpu().numpy(), (1, 2, 0)), interpolation='nearest')
                # plt.show()

                img_grid_real = torchvision.utils.make_grid(real_images[:32], normalize=True) 
                # show_img(img_grid_fake, f'{epoch}-fake')
                # show_img(img_grid_real, f'{epoch}-real')

                wandb.log({"fake_images": wandb.Image(img_grid_fake)})
                wandb.log({"real_images": wandb.Image(img_grid_real)})

            step += 1
            gen.train()
            critic.train()
  

In [ ]:

# config.lr = 1e-4
# config.batch_size = 64
# config.noise_dim = 100
# config.num_epochs = 100
# config.features_critic = 32
# config.features_g = 32
# config.critic_iters = 5
# config.lambda_gp = 10

sweep_config = {
  "name" : "my-sweep",
  "method" : "random",
  "parameters" : {
    "img_channels": {
        "values":[3]
    },
    "num_epochs" : {
      "values" : [100, 500,1000, 2000]
    },
    "lr" :{
        "values":[0.000001, 0.00001, 0.0001, 0.001]
    },
    "batch_size": {
        "values":[32]
    },
    "noise_dim":{
        "values":[100]
    },
    "features_critic": {
        "values":[32]
    },
    "features_g": {
        "values":[32]
    },
    "critic_iters": {
        "values":[1, 5, 10]
    },
    "lambda_gp": {
        "values":[1, 5, 10]
    }
  }
}
# train(sweep_config)


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="monet-gan")
wandb.agent(sweep_id, train, count=25)


Create sweep with ID: jvjtvylj
Sweep URL: https://wandb.ai/tbass134/monet-gan/sweeps/jvjtvylj


wandb: Agent Starting Run: t8h5kdli with config:
wandb: 	batch_size: 32
wandb: 	critic_iters: 10
wandb: 	features_critic: 32
wandb: 	features_g: 32
wandb: 	img_channels: 3
wandb: 	lambda_gp: 5
wandb: 	lr: 1e-06
wandb: 	noise_dim: 100
wandb: 	num_epochs: 500
wandb: Currently logged in as: tbass134 (use `wandb login --relogin` to force relogin)


{'batch_size': 32, 'critic_iters': 10, 'features_critic': 32, 'features_g': 32, 'img_channels': 3, 'lambda_gp': 5, 'lr': 1e-06, 'noise_dim': 100, 'num_epochs': 500}


Batch,5/10
Epoch,499/500
Loss Critic,-26.0001
loss G,12.2533


wandb: Agent Starting Run: ov66uq5a with config:
wandb: 	batch_size: 32
wandb: 	critic_iters: 1
wandb: 	features_critic: 32
wandb: 	features_g: 32
wandb: 	img_channels: 3
wandb: 	lambda_gp: 10
wandb: 	lr: 1e-05
wandb: 	noise_dim: 100
wandb: 	num_epochs: 1000


...

wandb: Agent Starting Run: 24n5y9pw with config:
wandb: 	batch_size: 32
wandb: 	critic_iters: 5
wandb: 	features_critic: 32
wandb: 	features_g: 32
wandb: 	img_channels: 3
wandb: 	lambda_gp: 1
wandb: 	lr: 0.0001
wandb: 	noise_dim: 100
wandb: 	num_epochs: 500


{'batch_size': 32, 'critic_iters': 5, 'features_critic': 32, 'features_g': 32, 'img_channels': 3, 'lambda_gp': 1, 'lr': 0.0001, 'noise_dim': 100, 'num_epochs': 500}


Batch,5/10
Epoch,499/500
Loss Critic,-146.1977
loss G,299.6875


wandb: Agent Starting Run: us45m4ta with config:
wandb: 	batch_size: 32
wandb: 	critic_iters: 10
wandb: 	features_critic: 32
wandb: 	features_g: 32
wandb: 	img_channels: 3
wandb: 	lambda_gp: 10
wandb: 	lr: 1e-05
wandb: 	noise_dim: 100
wandb: 	num_epochs: 2000


{'batch_size': 32, 'critic_iters': 10, 'features_critic': 32, 'features_g': 32, 'img_channels': 3, 'lambda_gp': 10, 'lr': 1e-05, 'noise_dim': 100, 'num_epochs': 2000}
